## Import All Necessary Modules And Setup Project
If you get any errors when importing these, ensure you run the command:

  $ python -m pip install -r requirements.txt

to install all necessary modules for this project. This command must be run from inside of this project directory.

It is recommended to use virtual environments for this project to ensure there is no conflicting package versions on your system.

Activate the virtual environment (if needed), run the pip install command, and then launch Jupyter Lab inside this project to get this project running.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
masterData=pd.read_csv("data/crime-housing-austin-2015.csv")
ZipcodeData=pd.read_csv("data/AustinZipCodes.csv")
#explore dataset
masterData.head()
#It receives an int or None (to print all the columns):
pd.set_option('display.max_columns', None)
masterData.head()

In [ ]:
masterData.Highest_NIBRS_UCR_Offense_Description.unique()


In [ ]:
ZipcodeData.head()

In [ ]:
#list out all the columns in a dataset
list(masterData)
#list unique values in a column "Council_District" of a dataset
masterData.Council_District.unique()
#clear rows that contains nan values
masterData=masterData.dropna()

In [ ]:
#type of a dataset columns
masterData.dtypes
#"council district" & "zip codes" are float, lets make it as int first
masterData=masterData.astype({"Council_District":'int',"Zip_Code_Crime":'int'})
#percent sign or dollar sign attached
#dataset['colname'] = dataset['colname'].str.replace('$', '').astype('float')
masterData['Medianhouseholdincome'] = masterData['Medianhouseholdincome'].str.replace('$', '').astype('float')
masterData['Unemployment'] = masterData['Unemployment'].str.replace('%', '').astype('int')

In [ ]:
#create pie plot for "Highest_NIBRS_UCR_Offense_Description"
sub_masterData=masterData[['Key','Zip_Code_Crime','District','Clearance_Status','Highest_NIBRS_UCR_Offense_Description',
                              'Medianhouseholdincome', 'Unemployment']]
sub_masterData=sub_masterData.rename(columns={'Zip_Code_Crime':'Zip Code','Highest_NIBRS_UCR_Offense_Description':'crime_type'})
display(sub_masterData)

In [ ]:
#aggregate the total number of each type of crimes
crime_types = sub_masterData.groupby('crime_type').agg(
    {'crime_type': 'count'})
crime_types=crime_types.rename(columns={'crime_type':'count'})
crime_types=crime_types.reset_index()
label=crime_types['crime_type']
plt.figure(figsize=(10,10))
plt.pie(data=crime_types, x='count',shadow=True)
plt.title('Texas Crime Results in 2015')
plt.legend(labels=label)
plt.savefig('pi_plot_categorical_crime.jpeg')

## main instructions
Your analysis must include a **number of statistical methods**. You must include `Pearson correlations` (be sure to report the **`p-value`**), _scatterplots_, _averages_, _standard deviations_, and a **t-test** (or **Mann-Whitney-U test**). The specific number of analyses is left up to you, but the contribution must be significant and your project report must give detailed justification and results for each analysis.